# О проекте
<a id='target5'></a>

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования могут быть учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.
  
  
## План:

Предобработка данных:
1. [Артефакты и пропуски](#target7) 
2. [Форматирование значений](#target8)
3. [Проверка](#target9)
 
Необходимо выяснить: 
1. [Есть ли зависимость между наличием детей и возвратом кредита в срок?](#target1)
2. [Есть ли зависимость между семейным положением и возвратом кредита в срок?](#target2)
3. [Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#target3)
4. [Как разные цели кредита влияют на его возврат в срок?](#target4)

Общие выводы:
1. [Резюме](#target6)

# Начало работы с файлом

Импортирую pandas, записываю полученные данные в переменную **df**, вывожу на экран информацию о таблице.

In [1]:
from pandas import DataFrame, Series
import pandas as pd
unrealistic = -777777

In [2]:
df = pd.read_csv('data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 1.6+ MB


От заказчика получил описание данных в таблице:

- **children** — количество детей в семье
- **days_employed** — общий трудовой стаж в днях
- **dob_years** — возраст клиента в годах
- **education** — уровень образования клиента
- **education_id** — идентификатор уровня образования
- **family_status** — семейное положение
- **family_status_id** — идентификатор семейного положения
- **gender** — пол клиента
- **income_type** — тип занятости
- **debt** — имел ли задолженность по возврату кредитов
- **total_income** — ежемесячный доход
- **purpose** — цель получения кредита


Ненулевых значений в столбцах **days_employed** и **total_income** одинаковое количество - 19351. В остальных столбцах ненулевых значений - 21525. В столбцах **days_employed** и **total_income** даны значения единообразного вида: формата float с шестью знаками после запятой.  
Очевидно, вычислением значений в этих столбцах занимался компьютер. Выведу фрагмент таблицы на экран:  

In [3]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


По фрагменту таблицы и по информации о таблице видно следующее:
- В **days_employed** есть отрицательные значения и есть странные шестизначные числа; 
- Значения в **days_employed** и **total_income** тяжелы для восприятия, их стоит отформатировать;
- Пропущенные значения следует искать в столбцах **days_employed** и **total_income**;
- Нужно привести значения к общему виду в столбце **education** и, возможно, в других столбцах.

# Предобработка данных в таблице
<a id='target7'></a>

## Отображение трудового стажа

In [4]:
print('В столбце days_employed:', end= '\n    ')
print('Значений всего -', df['days_employed'].count(), end= '\n    ')
print('Отрицательных значений -', df[ df['days_employed'] < 0]['days_employed'].count(), end= '\n    ')
print('Записей о тех, кто работает дольше чем живёт -', df[ (df['days_employed'] / 365) > df['dob_years'] ]['days_employed'].count())

В столбце days_employed:
    Значений всего - 19351
    Отрицательных значений - 15906
    Записей о тех, кто работает дольше чем живёт - 3445


Вычитая дату обработки заявки из даты отсчёта трудового стажа (когда даты записаны в формате UTC), вполне можно получить отрицательное значение. Как нетрудно увидеть по выводу выше, никаких положительных значений (кроме странных шестизначных) в столбце **days_employed** нет.

Чтобы рассмотреть аномалию с завышенным трудовым стажем, проиндексирую таблицу и выведу 5 строк на экран:

In [5]:
df[ (df['days_employed'] / 365) > df['dob_years'] ].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью


Кажется необычным, что во всех строках, где трудовой стаж завышен, клиентами являются пенсионеры. Проверю, какие категории клиентов встречаются в проиндексированном фрагменте:

In [6]:
print('Проблема с завышенным трудовым стажем замечена у следующих категорий клиентов:', 
      df[ (df['days_employed'] / 365) > df['dob_years'] ]['income_type'].value_counts(), sep= '\n')

Проблема с завышенным трудовым стажем замечена у следующих категорий клиентов:
пенсионер      3443
безработный       2
Name: income_type, dtype: int64


In [7]:
print('Пенсионеров всего в таблице - ', df[ df['income_type'] == 'пенсионер' ]['income_type'].count())
print('Безработных всего в таблице - ', df[ df['income_type'] == 'безработный' ]['income_type'].count())

Пенсионеров всего в таблице -  3856
Безработных всего в таблице -  2


In [8]:
print('Пропущенных значений в столбце days_employed у пенсионеров -', 
      df[ (df['income_type'] == 'пенсионер') ]['days_employed'].isna().sum())

Пропущенных значений в столбце days_employed у пенсионеров - 413


Удалось выяснить, что у одной части пенсионеров трудовой стаж не отображается (пропущенные значения), у другой - отображается шестизначное число, и ни у кого не отображается корректно. У всех безработных тоже отображается шестизначное число. Отталкиваясь от того, что вычислениями трудового стажа занимается компьютер, предполагаю, что информация о трудовом стаже этих клиентов поступает не в том формате, на который рассчитана формула для этого столбца. 

Рекомендую проверить, откуда и как в таблицу попадают данные о трудовом стаже пенсионеров и безработных. А пока что эти данные нужно заменить. Присвоив число _unrealistic_ (в переменную _unrealistic_ записано число -777777) вместо аномальных числовых значений, я смогу обезопасить себя от того, чтобы ненароком не вовлечь эти числа в какие-нибудь числовые расчёты (нереалистичные числовые значения легко отфильтровать, а если забыть это сделать - по выводу будет ясно, что нечто пошло не так). Удалять эти строки нельзя, они составляют почти 90% ото всех записей о пенсионерах. Заменить на _'unknown'_ можно, но возникнут проблемы, когда потребуется выполнить арифметическую или логическую операцию со значениями столбца. 
  
После замены умножу все значения столбца на -1, чтобы значения трудового стажа всё-таки стали положительными числами, а значения _unrealistic_, наоборот, отрицательными.

In [9]:
df.loc[ (df['days_employed'] / 365) > df['dob_years'], 'days_employed' ] = unrealistic * -1
    # пользуюсь тем, что деление NaN на число не вызывает исключения и возвращает False на любое логическое условие 
df.loc[:, 'days_employed'] *= -1

In [10]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,-777777.000000,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


## Удаление пропусков

Чтобы NaN'ы не мешали при форматировании числовых значений в столбцах, разобраться с ними лучше сейчас. Посчитаю их количество для всей таблицы и их долю от значений в этих таблицах:

In [11]:
print(df.isna().sum())
print(df.isna().mean())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64
children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64


Было обнаружено, что для некоторых категорий клиентов стаж вычисляется некорректно. Возможно, значения NaN тоже являются следствием некорректных вычислений данных о некоторых группах клиентов. Чтобы это проверить, создам таблицу, состоящую из строк, в которых встречается NaN - **df_just_nans**. Сгруппирую эту таблицу несколько раз по разным столбцам и посмотрю, встречается ли такая категория клиентов, которая содержит все значения NaN в этом столбце.

In [12]:
df_just_nans = df.loc[ df['total_income'].isna() | df['days_employed'].isna() ]
print(df_just_nans.groupby('income_type')['debt'].count())
print()
print(df_just_nans.groupby('family_status')['debt'].count())
print()
print(df_just_nans.groupby('education')['debt'].count())

income_type
госслужащий         147
компаньон           508
пенсионер           413
предприниматель       1
сотрудник          1105
Name: debt, dtype: int64

family_status
Не женат / не замужем     288
в разводе                 112
вдовец / вдова             95
гражданский брак          442
женат / замужем          1237
Name: debt, dtype: int64

education
ВЫСШЕЕ                   23
Высшее                   25
НАЧАЛЬНОЕ                 1
НЕОКОНЧЕННОЕ ВЫСШЕЕ       7
Начальное                 1
Неоконченное высшее       7
СРЕДНЕЕ                  67
Среднее                  65
высшее                  496
начальное                19
неоконченное высшее      55
среднее                1408
Name: debt, dtype: int64


Такой категории не нашлось ни в одной группировке. Тогда появление NaN, скорее всего, связано с тем, что присоединяемые столбцы **days_employed** и **total_income** содержали меньше записей о клиентах, чем хранилось в исходной таблице.

 Для экономии памяти удалю таблицу **df_just_nans**, содержащую строки с пропусками. Она больше не понадобится.  

In [13]:
%xdel df_just_nans

Информацию о стаже можно заполнить числом unrealistic - если для некоторого числа клиентов не будет данных об их стаже, это не скажется на возможности группировать, при необходимости, клиентов по стажу и получать корректные значения из таких группировок. А значения стажа, судя по целям, стоящим передо мной, понадобятся только для группировки. 

Другое дело со столбцом **total_income**: не охота исключать более двух тысяч строк (более 10% данных!) из столбца с данными для непосредственного анализа. Но, допустим, я вставил среднее значение.

Итак, нужно проявить осторожность, чтобы не внести в таблицу некорректную информацию. Напишу функцию, которая вычисляет два ожидаемых значения **total_income**: 
1. Средний доход (по медиане) для клиентов того же рода деятельности + того же возраста (плюс/минус пять лет). К сожалению, возможности обратиться к данным о клиентах с тем же стажем нет - во всех строках с пропусками в **income_type** также пропуски в стаже. Конечно, "госслужащий" в 50 лет может быть министром, а может доставлять квитанции - из среднего значений о лицах, о которых ничего не известно (даже если вычислять по медиане) нельзя сказать ничего определённого об ожидаемом доходе пятидесятилетнего человека, занятого в той же области. Отсюда сопоставление со следующим:
2. Средний доход (по медиане) для клиентов того же образования + рода деятельности. Люди с вышим образованием, вполне вероятно, зарабатывают в некоторой области в среднем больше, чем люди с тем же стажем, того же возраста, но без высшего образования. 

И сделаем следующее:

Если два вычисленных значения отличаются друг от друга на сумму, превышающую 20 тысяч у.е.*, то пропуск заполню значением _unrealistic_ (не буду брать ответственность за потенциально некорректные поправки), если меньше - то арифметическим средним этих двух значений. Пусть клиенту 50 лет, он госслужащий и у него среднее образование. Если в среднем госслужащие в 50 лет зарабатывают 220 тысяч, а люди со средним образованием в 50 лет зарабатывают 180 тысяч, то мы не можем предугадать, что было решающим фактором в трудовой судьбе именно этого клиента - то, что он работает в бюджетной сфере или то, что он получил среднее образование. Цена ошибки так же велика как и для остальных клиентов (неверная категоризация строчки), но в случаях наподобие этого из-за большого разброса средних значений особенно велик риск, что мы ошибёмся на такую сумму, из-за которой клиент попадёт не в свою категорию. 

Полагаться лишь на среднее значение по типу занятости + возрасту (да и даже по стажу, если бы он был!) было бы неправильно - к сожалению, в данной таблице не очень подробная категоризация по типу занятости - всего 4 вида занятости на большинство записей. Объединить все три параметра для подсчёта одного среднего значения тоже невозможно - в таблице слишком мало данных, которые будут удовлетворять такому условию.

*_20 тысяч у.е. разницы - это та разница, из-за которой клиент почти гарантированно будет впоследствии отнесён к другой категории по доходам_

In [14]:
inserted_mean_values = 0
inserted_unrealistic = 0
forced_unrealistic = 0
    # счётчик для проверки того, сколько каких значений присвоено

def count_expected_income(row):
    
    global inserted_mean_values
    global inserted_unrealistic
    global forced_unrealistic
    unrealistic = -777777
    
    if row['dob_years'] < 18:
        inserted_unrealistic += 1
        forced_unrealistic += 1
        return unrealistic
    # если возраст клиента меньше восемнадцати, то кредит бы ему не дали. 
    # это артефакт, с ними позже. можно считать, что возраст неизвестен. 
    
    f_expected_value = df[ (df['income_type'] == row['income_type']) &
                           (df['dob_years'] >= row['dob_years'] - 5 ) &
                           (df['dob_years'] <= row['dob_years'] + 5 ) ]['total_income']

    s_expected_value = df[ (df['education'].str.lower() == row['education'].lower()) &
                           (df['income_type'] == row['income_type']) ]['total_income']
    
    if f_expected_value.dropna().empty or s_expected_value.dropna().empty:
    # если один из срезов пустой или содержит одни пропуски, то данных для 
    # вычисления среднего недостаточно
        inserted_unrealistic += 1
        return unrealistic
    else: 
        f_expected_value = f_expected_value.median()
        s_expected_value = s_expected_value.median()
        
    if abs(f_expected_value - s_expected_value) < 20000:
        inserted_mean_values += 1
        return (f_expected_value + s_expected_value) / 2
    else:
        inserted_unrealistic += 1
        return unrealistic

In [15]:
# в этой ячейке работает тяжёлая функция, надо минуту подождать после запуска

df['days_employed'] = df['days_employed'].fillna(unrealistic)
df.loc[ df['total_income'].isna(), 'total_income' ] = df.loc[ df['total_income'].isna() ].apply(count_expected_income, axis = 1)
print('Добавлено средних значений в доходы:', inserted_mean_values)
print('Добавлено значений unrealistic в доходы: ', inserted_unrealistic, '. Из них {} - из-за некорректного возраста'.format(forced_unrealistic), sep = '')
print('Пропусков в стаже: {}, пропусков в доходе: {}'.format(df['days_employed'].isna().sum(), df['total_income'].isna().sum())) 

Добавлено средних значений в доходы: 1606
Добавлено значений unrealistic в доходы: 568. Из них 10 - из-за некорректного возраста
Пропусков в стаже: 0, пропусков в доходе: 0


## Изучение данных в остальных столбцах

В таблице, как было замечено ранее, допущены неявные дубли текстовых категорий. Значит, нужно проверить на них все столбцы с текстовыми категориальными переменными: **education**, **family_status**, **gender** и **income_type**. Вызову список уникальных значений в этих столбцах:

In [16]:
print()
print(df['education'].unique())
print()
print(df['family_status'].unique())
print()
print(df['gender'].unique())
print()
print(df['income_type'].unique())


['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']

['F' 'M' 'XNA']

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


Всё, в принципе, понятно, но стоит отдельно посмотреть на строку с 'XNA'

In [17]:
df[ df['gender'] == 'XNA' ]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Оставшиеся столбцы (кроме **purpose**) содержат числа. По идее, в этих столбцах не должно быть артефактов. Но поскольку остаётся неясным, получены эти числа из пользовательского ввода или из проверенной базы данных, стоит предположить худшее и проверить их на наличие артефактов тоже. Чтобы это сделать, упорядочу числа в тех столбцах, где множество возможных значений слишком велико для вывода, и выведу несколько значений с конца и с начала. Для тех столбцов, где множество возможных значений не должно быть слишком велико, вызову список уникальных значений:

In [18]:
print(df.sort_values(by = 'children', ascending = False)['children'].head(3))
print(df.sort_values(by = 'children', ascending = False)['children'].tail(3))
print()
print(df.sort_values(by = 'dob_years', ascending = False)['dob_years'].head(3))
print(df.sort_values(by = 'dob_years', ascending = False)['dob_years'].tail(3))

19562    20
11715    20
3302     20
Name: children, dtype: int64
21140   -1
19100   -1
15147   -1
Name: children, dtype: int64

8880    75
4895    74
2557    74
Name: dob_years, dtype: int64
2469     0
1890     0
14514    0
Name: dob_years, dtype: int64


In [19]:
print(df['debt'].unique())
print(df['education_id'].unique())
print(df['family_status_id'].unique())

[0 1]
[0 1 2 3 4]
[0 1 2 3 4]


Наличие задолженности и разнообразные id отображаются корректно. А вот что имели в виду те клиенты, которые указали, что у них 20 детей или что им 0 лет, я буду разобраться. Пока просто уточню, сколько у меня странных значений, связанных с детьми и с возрастом, и какие это значения:

In [20]:
print(df[ df['children'] > 4 ]['children'].value_counts())
print()
print(df[ df['children'] < 3 ]['children'].value_counts())
print()
print(df[ df['dob_years'] < 20 ]['dob_years'].value_counts())

20    76
5      9
Name: children, dtype: int64

 0    14149
 1     4818
 2     2055
-1       47
Name: children, dtype: int64

0     101
19     14
Name: dob_years, dtype: int64


Проверю таблицу целиком на наличие грубых дублей.  
Искать грубые дубли в столбцах не нужно - ни в одном столбце нет таких значений, которые должны быть уникальными

In [21]:
df.duplicated().sum()

54

Чтобы посмотреть на дубликаты "глазами", передам методу `duplicated()` параметр `keep = False`, с помощью которого смогу вывести не только последний встретившийся дубль, а все дублированные строки. 
  
Добавлю произвольное условие к методу индексации, чтобы сократить количество строк в выводе. Метод `.head()` здесь не подойдёт: он выводит порядковые значения, а в таблице дубли не обязательно следуют друг за другом по порядку.

In [22]:
df[ (df.duplicated(keep = False) == True) & (df['purpose'] == 'свадьба') ].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2596,0,-777777.0,60,среднее,1,гражданский брак,1,F,пенсионер,0,117963.442074,свадьба
3690,1,-777777.0,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,-777777.000000,свадьба
4182,1,-777777.0,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,-777777.000000,свадьба
4529,0,-777777.0,57,среднее,1,гражданский брак,1,M,пенсионер,0,117520.474363,свадьба
4851,0,-777777.0,60,среднее,1,гражданский брак,1,F,пенсионер,0,117963.442074,свадьба
7995,0,-777777.0,57,среднее,1,гражданский брак,1,F,пенсионер,0,117520.474363,свадьба
15273,0,-777777.0,57,среднее,1,гражданский брак,1,F,пенсионер,0,117520.474363,свадьба
20116,0,-777777.0,57,среднее,1,гражданский брак,1,M,пенсионер,0,117520.474363,свадьба


Попробуем сосчитать дубли, у которых в столбце **'days_employed'** и **'total_income'** числа, а не пропуск. Если таковых не найдётся, то для меня станет более ясной природа дублей в этой таблице.

In [23]:
df[ (df.duplicated() == True) & 
   ((df['days_employed'] != unrealistic) & (df['total_income'] != unrealistic)) ]['days_employed'].count()

0

Итак, работы много. Формулирую найденные проблемы:
- **Образование.** Есть 5 значений, которые может принимать переменная **'education_id'** и есть ровно пять уникальных значений переменной **'education'**: _'высшее', 'среднее', 'начальное', 'неоконченное высшее', 'ученая степень'_. Значит, между этими двумя переменными в самом деле возможно прямое соответствие (как это следует из их названий). Остаётся заменить неявные дубли в столбце **'education'**. Очевидно, что этого можно достичь просто опустив регистр всех значений столбца.
- **Семейный статус.** В строчке _'Не женат / не замужем'_ для удобства следует заменить заглавную букву строчной, в остальном проблем никаких.
- **Пол 'XNA'.** XNA может означать человека, не определившегося с гендером. Наш ли это случай? В таблице есть только одна запись со значением 'XNA'. Клиент молод, живёт с другом или подругой и не имеет детей, так что - вполне возможно. Если так, то запись не нуждается в исправлении. Неудобств так же не возникнет, поскольку в этом проекте нам не потребуется индексировать таблицу по полу. 
- **Количество детей.** Нет ни одного клиента, у кого было бы больше 5 детей, но меньше 20. Зато тех, у кого их ровно 20 - целых 76 штук. Очевидно, дело в ошибке при вводе, когда по какой-то причине был выбран крайний вариант из возможных. Но что означает запись -1 ребёнок? Вряд ли "-1" было среди вариантов ввода. Возможно, некоторые клиенты при заполнении анкеты желали поставить тире (а система восприняла всё вместе как отрицательное число). Но если на 4818 анкет с "1" ребёнком приходится 47 анкет с "-1" ребёнком, то на 2055 анкет с "2" детьми (а мы проверили, что их именно столько) должно было бы приходиться приблизительно 20 анкет с "-2" детьми. Но из отрицательных значений в таблице есть только "-1". Поэтому как "20", так и "-1" я никак не могу пропустить, и с моей стороны будет правильно их заменить на _unrealistic_. 
- **Возраст клиента.** Нет ни одного клиента, кому было бы меньше 19, но больше 0. Зато тех, кому ровно 0 - целых 101 штука. Аналогичная ситуация, значение "0" нужно заменить на _unrealistic_. 
- **Грубые дубли.** С дубликатами требуется принять сложное решение, требуется выяснить их природу. Ведь если две одинаковые строки содержат информацию о двух разных людях (информация банально совпадает), то удаление даже одной из них приведёт к искажениям в выводах. Если же я оставлю одинаковые строки, а окажется, что это именно дубли - то есть, две записи об одном человеке - то на анализ попадёт больше значений, чем должно.  
    Оттолкнусь от следующего рассуждения: если в дублированных строках совпадают значения нестандартного формата (например, "ВЫСШЕЕ" и "ВЫСШЕЕ"), то маловероятно, чтобы два разных человека внесли одинаково-нестандартные значения - такие дубли можно сократить. Пусть мы удаляем только эти дубли. Тогда во всех остальных случаях мы имеем полные совпадения по кол-ву детей, возрасту, образованию, полу, семейному статусу и цели, для которой берётся кредит. По большому счёту, такие совпадения тоже маловероятны. Кроме того, все до последней дублированные пары относятся к тем строкам, в которых были пропущенные значения и артефакты - а это убеждает в той мысли, что появление дублей связано с какой-то ошибкой. В итоге, можно со спокойной совестью удалить все найденные дубликаты, а информацию о них передать для доработки. Даже если среди удалённых строк найдутся одна-две пары, которые относились к разным людям, итоговая погрешность будет сравнительно невелика.
<a id='target8'></a>

## Форматирование значений

Удаляю дубли, переназначаю индексы в таблице, удаляю артефакты из столбцов **'children'**, **'dob_years'**, корректирую все вхождения _'Не женат / не замужем'_, опускаю в нижний регистр все значения столбца **'education'**:

In [24]:
df = df.drop_duplicates().reset_index(drop = True)
print('Дублей в таблице -', df.duplicated().sum())
df.loc[ (df['children'] == -1) | (df['children'] == 20), 'children' ] = unrealistic
df.loc[ df['dob_years'] == 0, 'dob_years' ] = unrealistic
df.loc[ df['family_status'] == 'Не женат / не замужем', 'family_status' ] = 'не женат / не замужем'
df['education'] = df['education'].str.lower()

Дублей в таблице - 0


Для видоизменения данных в столбцах **'days_employed'** и **'total_income'** я написал следующие функции:

In [25]:
def months_instead_of_days(value):
    """
    Принимает очередное значение столбца days_employed. Если принимает положительное
    число, то функция возвращает объект int, где число означает кол-во месяцев стажа.
    Если отрицтельное (такие в этом столбце только пропуски), то возвращает его же.
    Привычнее было бы перевести в годы, но годам следовало бы передать десятичную часть -
    например, 12,74 лет стажа, но объекты int предпочтительнее чем float.
    """
    if value > 0:
        months = (int(value) // 365) * 12
        return months
    return value

In [26]:
def income_in_thousands(income):
    """
    Принимает очередное значение столбца total_income. Пропуски функция возвращает 
    без изменений. Для остальных значений возвращает результат целочисленного деления
    на тысячу. Напр., число 83461.20989057 на выходе станет числом 83, что будет 
    означать, что ежемесячный доход клиента - 83 тысячи у.е.
    """
    if income != unrealistic:
        return int(income // 1000)
    return income

Применяю пользовательские функции к столбцам, для которых они написаны:

In [27]:
df['days_employed'] = df['days_employed'].apply(months_instead_of_days)
df['total_income'] = df['total_income'].apply(income_in_thousands)

Меняю значения этих столбцов с float на int для скорости и точности будущих вычислений. Благодаря тому что пропуски были заполнены целыми числами, это можно сделать без проблем. 

In [28]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

Название столбца **days_employed** больше не отражает его содержание, ведь я перевёл дни в месяцы. Актуализирую названия столбцов:

In [29]:
axis = ['children', 'months_employed', 'dob_years', 'education', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose']
df.set_axis(axis, axis = 'columns', inplace = True)
df.columns

Index(['children', 'months_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

<a id='target9'></a>

## Проверка

Я закончил форматировать данные в таблице, проверю результаты:

In [30]:
print('Максимальный и минимальный стаж, мес.:')
print(df[ df['months_employed'] != unrealistic ]['months_employed'].max())
print(df[ df['months_employed'] != unrealistic ]['months_employed'].min())
print()
print('Максимальный и минимальный месячный доход, тыс. у.е.:')
print(df[ df['total_income'] != unrealistic ]['total_income'].max())
print(df[ df['total_income'] != unrealistic ]['total_income'].min())
print()
print('Максимальный и минимальный возраст, лет:')
print(df[ df['dob_years'] != unrealistic ]['dob_years'].max())
print(df[ df['dob_years'] != unrealistic ]['dob_years'].min())
print()
print('Максимальное и минимальное количество детей, шт.:')
print(df[ df['children'] != unrealistic ]['children'].max())
print(df[ df['children'] != unrealistic ]['children'].min())
    # Надеюсь получить адекватные цифры

Максимальный и минимальный стаж, мес.:
600
0

Максимальный и минимальный месячный доход, тыс. у.е.:
2265
20

Максимальный и минимальный возраст, лет:
75
19

Максимальное и минимальное количество детей, шт.:
5
0


In [31]:
print(df['education'].unique())
print(df['family_status'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21471 non-null  int64 
 1   months_employed   21471 non-null  int32 
 2   dob_years         21471 non-null  int64 
 3   education         21471 non-null  object
 4   education_id      21471 non-null  int64 
 5   family_status     21471 non-null  object
 6   family_status_id  21471 non-null  int64 
 7   gender            21471 non-null  object
 8   income_type       21471 non-null  object
 9   debt              21471 non-null  int64 
 10  total_income      21471 non-null  int32 
 11  purpose           21471 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.4+ MB


In [33]:
df.head()

,children,months_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,276,42,высшее,0,женат / замужем,0,F,сотрудник,0,253,покупка жилья
1,1,132,36,среднее,1,женат / замужем,0,F,сотрудник,0,112,приобретение автомобиля
2,0,180,33,среднее,1,женат / замужем,0,M,сотрудник,0,145,покупка жилья
3,3,132,32,среднее,1,женат / замужем,0,M,сотрудник,0,267,дополнительное образование
4,0,-777777,53,среднее,1,гражданский брак,1,F,пенсионер,0,158,сыграть свадьбу


Как явных, так и неявных дублей в таблице не осталось. Артефакты найдены и обработаны. Пропущенных значений в таблице нет. Значения в столбцах **years_employed** и **total_income** теперь читаются намного легче. Кроме того, для них установлен тип данных _int_. Всё в порядке, данные можно анализировать.

# Анализ данных

Я собираюсь изучить взаимосвязи одной переменной из предоставленной таблицы (значение столбца **'debt'**) с другими перменными (значениями других столбцов). Значит, сводная таблица, какие бы я взаимосвязи не изучал, будет примерно похожа: её индексами будут группы значений столбца **'debt'**, её столбцами будут группы значений некоего произвольно взятого столбца. Аргументу `values=` можно передать название любого столбца, так как затем мы укажем `aggfunt = 'count'`, а после применения `.count()` окажутся неподсчитанными только те принадлежащие обеим группам записи, у которых в переданном столбце стоит пропуск. Но пропусков нет ни в одном столбце, поэтому аргумент `values=` может быть любым.

Напишу функцию, создающую сводную таблицу по этим условиям:

In [34]:
def create_pivot_table(column):

    # функция возьмёт df как глобальную переменную 
    pivot = df.pivot_table(index = 'debt',
                               columns = column,
                               values = 'gender',
                               aggfunc = 'count')
    
    problematic_rate = dict((x, '{:.2%}'.format(pivot[x][1] / (pivot[x][0] + pivot[x][1]))) for x in pivot.columns)
    for key in problematic_rate.keys():
        if problematic_rate[key] == 'nan%':
            problematic_rate[key] = '-'
            # заменяю на прочерк те значения словаря, которые получаются в результате
            # деления NaN или на NaN, для красоты
            
    pivot = pivot.append(problematic_rate, ignore_index = True)
    return pivot

Для каждого столбца сводной таблицы функция дополнительно вычисляет долю кредитов, по которым имела место задолженность, создаёт словарь **problematic_rate**, где ключом является название столбца, а значением - доля проблемных кредитов в этом столбце. С помощью метода pandas `.append()` я присоединяю строчку с **problematic_rate** к сводной таблице (чтобы сделать это без проблем, был создан именно словарь, а не другая структура). Функция возвращает сводную таблицу в таком виде, когда её уже можно анализировать.
<a id='target1'></a>

## Наличие детей: влияние на платёжеспособность клиента

Данные в столбце **'children'** сами по себе сгруппируются по шести категориям (пять числовых значений столбца + _unrealistic_), что вполне приемлимо. Раз подготавливать создание сводной таблицы не нужно, сразу создам её:

In [35]:
create_pivot_table('children')

,-777777,0,1,2,3,4,5
0,114,13044,4365,1858,303,37,9
1,9,1063,444,194,27,4,NaN
2,7.32%,7.54%,9.23%,9.45%,8.18%,9.76%,-


Бездетных заёмщиков больше, чем всех остальных вместе. Остаётся ориентироваться только на долю проблемных кредитов от общего числа кредитов для каждой группы заёмщиков. Данные для клиентов с 4 и 5 детьми лучше оставить в стороне, пока что таких клиентов слишком мало. Из данных в столбце **-777777** никакого вывода не следует. Настрою срёз: 

In [36]:
create_pivot_table('children').loc[:, [0, 1, 2, 3]]

,0,1,2,3
0,13044,4365,1858,303
1,1063,444,194,27
2,7.54%,9.23%,9.45%,8.18%


Судя по таблице, по кредитам тяжелее расплачиваться, когда у тебя есть дети. Однако, не похоже, чтобы большее или меньшее количество детей сильно сказывалось на платёжеспособности клиента. Вероятно, дело в финансовой грамотности: если денег не хватает, то либо следующего ребёнка не заводят, либо не рискуют брать кредиты.
<a id='target2'></a>

## Семейное положение: влияние на платёжеспособность клиента

В таблице есть два столбца, хранящие информацию о семейном положении: **'family_status'** и **'family_status_id'**. В каждом встречаются лишь шесть уникальных значений, каждому значению из одного столбца соответсвует некоторое значение из другого. Без разницы, какой столбец я передам в сводную таблицу - цифры будут одинаковыми. Но если я передам **'family_status'**, то по названиям столбцов будет проще ориентироваться.

In [37]:
create_pivot_table('family_status')

family_status,в разводе,вдовец / вдова,гражданский брак,женат / замужем,не женат / не замужем
0,1110,896,3775,11413,2536
1,85,63,388,931,274
2,7.11%,6.57%,9.32%,7.54%,9.75%


С первого взгляда видно, что одинок человек или нет - разницы никакой. Чтобы найти более тонкий критерий, попробуем посмотреть на отсортированные категории по убыванию их "проблемности": 
  
Не женат ---> Гражданский брак ---> Женат ---> В разводе ---> Вдовец  
Не женат ---> Гражданский брак ---> Женат ---> В разводе / Вдовец
  
Бросается в глаза, что эта прогрессия описывает не только благонадёжность должника, но и стадии в развитии отношений.
Можно описать это так: чем дальше клиент заходит в отношениях с близким человеком, тем надёжнее он становится для банка. Но вследствие чего? 

Попробую проверить самую банальную гипотезу: по мере развития отношений человек и сам постепенно взрослеет. Если так, то можно говорить, что не вдовцы самые надёжные должники, а люди настолько взрослые, что многие из них к своему возрасту становятся вдовцами. 
  
Чтобы это проверить, сгруппирую таблицу по семейному положению и вычислю средний возраст для каждой группы. Для этого применю `mean()` к столбцу **'dob_years'** фрагмента таблицы без значений _unrealistic_.

In [38]:
df[ df['dob_years'] != unrealistic ].groupby('family_status')['dob_years'].mean()

family_status
в разводе                45.902110
вдовец / вдова           56.799790
гражданский брак         42.321101
женат / замужем          43.722489
не женат / не замужем    38.583393
Name: dob_years, dtype: float64

Чуть выше была приведена прогрессия. Заменим категории клиентов средним возрастом в этой категории:
  
38 лет ---> 42 года ---> 43 года ---> 45 лет ---> 56 лет
  
Гипотеза поддтвердилась - не столько семейное положение влияет на благонадёжность заёмщика, сколько его возраст. Уверен, что если бы я категоризировал таблицу по возрасту, то увидел бы именно это.
<a id='target3'></a>

## Уровень дохода: влияние на платёжеспособность клиента 

In [39]:
length_of_income = len(df[ df['total_income'] != unrealistic ])
print('Имеется {} строк с информацией о доходах'.format(length_of_income))

Имеется 20910 строк с информацией о доходах


In [40]:
print('Эти данные можно разбить на равные группы, содержащие', length_of_income / 5, length_of_income / 6, length_of_income / 8, 'или', length_of_income / 10, 'строк')

Эти данные можно разбить на равные группы, содержащие 4182.0 3485.0 2613.75 или 2091.0 строк


Шесть групп по 3485 элементов в каждой - оптимальное разибиение. Обширное, и в то же время группы содержат достаточно элементов для того, чтобы делать о них обобщённые выводы. Использую функцию `pd.qcut()` чтобы разбить значения столбца **total_income** (без значений unrealistic) на равные по числу элементов группы. Посредством аргумента `labels =` передам названия категорий вместо стандартных интервалов. 
  
Опираться на квантили в данном случае лучше, чем на равные промежутки между максимальным и минимальным значением - доходы большей части клиентов будут принадлежать первым нескольким промежуткам из-за одного клиента, который зарабатывает как Билл Гейтц. Результаты присвоения запишу в новый столбец **'income_id'**

In [41]:
df['income_id'] = pd.qcut((df[ df['total_income'] != unrealistic ]['total_income']), 6, labels = ('very low', 'low', 'average', 'above average', 'high', 'very high'))
df['income_id'].unique()

['very high', 'low', 'above average', 'average', 'very low', 'high', NaN]
Categories (6, object): ['very low' < 'low' < 'average' < 'above average' < 'high' < 'very high']

Поскольку значениям _unrealistic_ не была присвоена никакая категория, в столбце **'income_id'** в строках с ними появились пропуски. Чтобы в результирующем столбце типа _category_ заполнить пропуски значением _'unknown'_, передам сначала _'unknown'_ как категорию. Затем вызову метод `.fillna()`

In [42]:
df['income_id'] = df['income_id'].cat.add_categories('unknown')
df['income_id'] = df['income_id'].fillna('unknown')

In [43]:
df['income_id'].value_counts()

above average    3570
very low         3564
low              3489
very high        3466
high             3416
average          3405
unknown           561
Name: income_id, dtype: int64

Вызову сводную таблицу для столбца **'income_id'**

In [44]:
create_pivot_table('income_id')

,very low,low,average,above average,high,very high,unknown
0,3280,3199,3100,3252,3141,3226,532
1,284,290,305,318,275,240,29
2,7.97%,8.31%,8.96%,8.91%,8.05%,6.92%,5.17%


Казалось, что клиенты с очень малым заработком будут очень часто иметь задолженность за кредит. Оказалось наоборот - у таких клиентов одни из лучших показатели. У клиентов с большим заработком, ожидаемо, проблем с выплатой задолженности, как правило, не возникает.

То, что у исключённых из всех категорий клиентов доля задолженностей сильно отличается (невзирая на то, что их число сравнительно мало) - говорит о том, что сортировка по функции `count_expected_value` не была напрасной. Неподтверждённые данные могли занизить ключевой показатель в одной или нескольких категориях.
  
Но из того, что выводы сводной таблицы можно схематично описать, не следует с необходимостью, что в выводах заложена какая-либо закономерность. В этих данных, например, никакой явной закономерности не видно. 

Более перспективным могло бы быть исследование данных не по доходу, а по типу занятости (студент, пенсионер, сотрудник компании и т.д.). Попробую вызвать таблицу с двойной категоризацией: по уровню дохода и по типу занятости. Так как такая таблица будет излишне растянута в ширину, для удобства транспонирую её методом `.T` и выведу для ознакомления лишь несколько категорий.

In [45]:
create_pivot_table(['income_id', 'income_type']).T.loc[['very low', 'above average', 'very high'], :]

0    1       2
income_id     income_type                       
very low      безработный       NaN    1       -
              в декрете         NaN    1       -
              госслужащий       208   18   7.96%
              компаньон         387   36   8.51%
              пенсионер        1041   53   4.84%
              сотрудник        1644  175   9.62%
above average госслужащий       276   14   4.83%
              компаньон         935   84   8.24%
              пенсионер         452   26   5.44%
              сотрудник        1589  194  10.88%
very high     госслужащий       233   12   4.90%
              компаньон        1197   76   5.97%
              пенсионер         335   18   5.10%
              предприниматель     1  NaN       -
              сотрудник        1460  134   8.41%

Рискованно делать выводы из данных, собранных для таких малых по количеству групп. Но нельзя игнорировать то, что во всех группах складывается похожая картина: сотрудники компаний оказываются самыми ненадёжными должниками, а госслужащие - самыми надёжными. Это говорит о том, что если нужно использовать данные о доходах клиента как критерий для кредитного скоринга, то более удачным критерием послужит не уровень доходов, а тип занятости.
<a id='target4'></a>

## Цели кредита: влияние на платёжеспособность клиента

Столбца **'purpose'** я до сих пор никак не касался. Но можно было заметить, что в нём много уникальных значений, и что некоторые из них, как то _"операции с жильём"_ или _"покупка жилой недвижимости"_, означают примерно одно и то же. 
  
Импортирую нужные инструменты из библиотек pymystem3 и nltk:

In [46]:
from pymystem3 import Mystem
from nltk import SnowballStemmer
m_m = Mystem()
s_s = SnowballStemmer('russian')

Лемматизирую значения столбца _purpose_. Создам множество со значениями столбца, чтобы, во-первых, исключить функцией `set()` повторы, а во-вторых, чтобы создать такой объект, который можно передать методу `.join()`. Метод `.join()`, в свою очередь, нужен для того, чтобы создать одну строчку вместо многих - со многими строчками функция `m_m.lemmatize()` будет работать непозволительно долго (приходится принимать во внимание ограниченные ресурсы моего ПК и связанную с ними устаревшую версию библиотеки pymystem3).

In [47]:
key_words = set(m_m.lemmatize(' '.join(set(df['purpose']))))

Функция `set()` возвращает множество уникальных лемм. Удалю из этого множества пробелы, предлоги и проч. с помощью `.remove()`. В конце выведу удалённые элементы на экран (чтобы убедиться, что не удалил лишнего). Затем выведу сами леммы.

In [48]:
print('Удалены:', end=' ')
deleted_from_set = []

for x in key_words.copy():
# поскольку речь об удалении элементов из коллекции, по которой
# я одновременно интерируюсь, необходимо итерироваться по копии
    if len(x) < 4:
        key_words.remove(x)
        deleted_from_set.append(x)
        
print([r'{}'.format(x) if ('\n' in x or '\t' in x) else x for x in deleted_from_set])

Удалены: ['со', 'с', 'для', '\n', ' ', 'на']


In [49]:
print(key_words)

{'жилье', 'подержать', 'приобретение', 'автомобиль', 'подержанный', 'сделка', 'проведение', 'покупка', 'жилой', 'свой', 'сдача', 'дополнительный', 'высокий', 'свадьба', 'семья', 'коммерческий', 'профильный', 'образование', 'сыграть', 'операция', 'заниматься', 'получение', 'ремонт', 'собственный', 'строительство', 'недвижимость'}


Чтобы прояснить, в каком контексте используются леммы, напишу функцию, которая принимает лемму и выводит на экран уникальные значения столбца **purpose** с этой леммой. Это важный этап, поскольку, например, лемма _'строительство'_ может встречаться как в выражении _'на строительство дома'_, так и в выражении _'на строительство демократии в родной стране'_. Тогда я проигнорирую лемму _'строительство'_ и буду искать другие ключевые слова.

In [50]:
unique = df['purpose'].unique()

def where_can_i_find(lemma):
    print('Значения со словом "{}":'.format(lemma))
    print('   ', end='')
    for x in unique:
        if s_s.stem(lemma) in s_s.stem(x):
        # функция возьмёт unique и s_s как глобальные переменные
            print(x, end=', ')
    print()
    print()

Запущу эту функцию в цикле, итерируясь по всем ранее найденным леммам

In [51]:
for x in key_words:
    where_can_i_find(x)

Значения со словом "жилье":
   покупка жилья, операции с жильем, покупка жилья для семьи, покупка жилой недвижимости, строительство жилой недвижимости, жилье, покупка своего жилья, покупка жилья для сдачи, ремонт жилью, 

Значения со словом "подержать":
   на покупку подержанного автомобиля, сделка с подержанным автомобилем, 

Значения со словом "приобретение":
   приобретение автомобиля, 

Значения со словом "автомобиль":
   приобретение автомобиля, на покупку подержанного автомобиля, на покупку своего автомобиля, сделка с подержанным автомобилем, автомобиль, свой автомобиль, сделка с автомобилем, на покупку автомобиля, 

Значения со словом "подержанный":
   на покупку подержанного автомобиля, сделка с подержанным автомобилем, 

Значения со словом "сделка":
   сделка с подержанным автомобилем, сделка с автомобилем, 

Значения со словом "проведение":
   на проведение свадьбы, 

Значения со словом "покупка":
   покупка жилья, покупка жилья для семьи, покупка недвижимости, покупка коммер

Есть слова, у которых при склонении видоизменяется корень. Можно увидеть - для некоторых слов функция `s_s.stem()` не смогла найти вхождения в столбце: _"заниматься"_ и _"высокий"_. Вызову ещё раз функцию, но передам ей другие корни этих каверзных слов.

In [52]:
where_can_i_find('занят')
where_can_i_find('выс')

Значения со словом "занят":
   заняться образованием, заняться высшим образованием, 

Значения со словом "выс":
   получение высшего образования, высшее образование, заняться высшим образованием, 



Теперь составлю список лемм-индикаторов конкретно для этой таблицы, на основании которых можно отнести любую запись к определённой категории:
- жилой + недвижимость + операция - коммерческий - сдача - ремонт --> **`жилая недвижимость`**  
    _Выделить категорию клиентов, берущих кредит для покупки жилья, можно так: из множеств записей, в которые входят леммы "жилой", "недвижимость" и "операция" нужно вычесть подмножество записей, в которые входят леммы "коммерческий", "сдача" (покупка недвижимости с целью заработка), "ремонт" (не покупка, а ремонт недвижимости), "строительство" (не покупка, а строительство)_.
- коммерческий, сдача --> **`коммерция с недвижимостью`**
- строительство --> **`строительство`**
- ремонт --> **`ремонт`**
- образование --> **`образование`**  
    _Есть много ключевых слов, однозначно указывающих на категорию "образование" (получение, высокий, профильный, заниматься...), но достаточно одного - "образование", так как нет таких ячеек, где речь шла бы об образовании, а самого слова "образование" не было бы_.
- сделка, автомобиль --> **`автомобиль`**
- сыграть, свадьба, проведение --> **`свадьба`** 
- **`прочее`**
  
Теперь напишу функцию, которая принимает строчку таблицы и возвращает категорию клиента по цели кредита - **'purpose_id'**.
Применю эту функцию к списку уникальных значений, чтобы создать таблицу соответствия **'purpose_id'**.

In [53]:
def assign_purpose_id(row, s_s = s_s):
    purpose = row['purpose']
    if s_s.stem('жилой') in purpose or s_s.stem('недвижимость') in purpose or s_s.stem('операция') in purpose:
        if s_s.stem('коммерческий') in purpose or s_s.stem('сдача') in purpose:
            return 'bying a place to rent'
        elif s_s.stem('ремонт') in purpose:
            return 'home improvement'
        elif s_s.stem('строительство') in purpose:
            return 'housebuilding'
        else:
            return 'bying a place to live'
    elif s_s.stem('образование') in purpose:
        return 'education'
    elif s_s.stem('сделка') in purpose or s_s.stem('автомобиль') in purpose:
        return 'bying a car'
    elif s_s.stem('сыграть') in purpose or s_s.stem('свадьба') in purpose or s_s.stem('проведение') in purpose:
        return 'making a wedding'
    else:
        return 'other'

Чтобы оптимизировать категоризацию, лучше применить функцию построчно не ко всей таблице, а к таблице уникальных значений - уникальных значений намного меньше. Получится таблица соответствия, которую можно будет присоединить к основной таблице с помощью метода `.merge()`.

Создаю таблицу соответствия, применяю к ней написанную функцию и вызываю первые пять строчек для проверки.

In [54]:
mapping_table = pd.DataFrame(data = unique, columns = ['purpose'])
mapping_table['purpose_id'] = mapping_table.apply(assign_purpose_id, axis = 1)
mapping_table.head()

,purpose,purpose_id
0,покупка жилья,bying a place to live
1,приобретение автомобиля,bying a car
2,дополнительное образование,education
3,сыграть свадьбу,making a wedding
4,операции с жильем,bying a place to live


Склеиваю две таблицы и для проверки подсчитываю количество всех значений **'purpose_id'**

In [55]:
df = df.merge(mapping_table, on = 'purpose')
df['purpose_id'].value_counts()

bying a place to live    6364
bying a car              4308
education                4014
making a wedding         2335
bying a place to rent    1964
housebuilding            1879
home improvement          607
Name: purpose_id, dtype: int64

Отлично! С помощью ключевых слов удалось категоризировать все строки, ни одного _'other'_ в **purpose_id**. Вызываю сводную таблицу:

In [56]:
create_pivot_table('purpose_id')

purpose_id,bying a car,bying a place to live,bying a place to rent,education,home improvement,housebuilding,making a wedding
0,3905,5912,1813,3644,572,1735,2149
1,403,452,151,370,35,144,186
2,9.35%,7.10%,7.69%,9.22%,5.77%,7.66%,7.97%


Интерпретация данных:
- Хуже всего возвращают кредиты те, кто потратил их на покупку машины и на образование. Ничего удивительно, от автомобилей одно раззорение. А студентам негде зарабатывать, да и некогда - они кредит брали чтобы учиться, а не работать. 
- Свадьба на кредитные средства - довольно легкомысленная трата. Но, видимо, семейная жизнь образумливает, да и родственники обычно помогают - такие кредиты в "середине" рейтинга. 
- Меньше всего проблем банку доставляют клиенты, чей кредит связан с недвижимостью. Можно предположить, что средств для покупки недвижимости требуется на порядок больше, чем на другие цели, и к такому кредиту клиенты подходят осознанно и с расчётом. Крупная задолженность не позволяет им расслабиться до тех пор, пока они полностью не рассчитаются. Притом реже всего задолжают банку те, кто на кредит купил жилплощадь для жизни, а не для коммерции - зарабатывать на недвижимости, похоже, у большинства не получается. 
<a id='target6'></a>

# Резюме

Выполнение проекта состояло из следующих этапов:
1. Получили первоначальное представление о том, над чем предстоит работать;
2. Выявили проблему с отображением трудового стажа у пенсионеров и безработных;
3. Удалили артефакты из данных о рабочем стаже;
4. Выявили проблему с присоединением к исходной таблице данных о стаже и доходах более чем двух тысяч клиентов;
5. Где оказалось возможным, рассчитали наиболее вероятный доход клиента и вставили вместо пропуска;
6. Избавились от всех оставшихся пропусков;
7. В двух столбцах нашли артефакты, в двух других - неявные дубли;
8. Нашли грубые дубли в таблице и приняли решение их удалить, удалили;
9. Видоизменили значения в столбцах со стажем и доходом и изменили в них формат данных;
10. Написали функцию, которая возвращает сводную таблицу и вычисляет долю проблемных кредитов;
11. Исследовали в интересах кредитора наличие детей у клиентов банка
12. ... их семейное положение
13. Категоризировали клиентов по уровню доходов
14. ... их уровень доходов
15. Категоризировали клиентов по целям, на которые они берут кредит
16. ... заявленные цели

Вызову всё сводные таблицы ещё раз и резюмирую выводы.

**По наличию детей**

In [57]:
create_pivot_table('children').loc[:, [0, 1, 2, 3]]

,0,1,2,3
0,13044,4365,1858,303
1,1063,444,194,27
2,7.54%,9.23%,9.45%,8.18%


По кредитам тяжелее расплачиваться тем, у кого есть дети. Лучше, когда у клиентов нет детей. Но если дети есть, то по большому счёту, уже не важно, сколько их - процент проблемных кредитов для клиентов с детьми держится на одном уровне. Нужно отметить, что данных о клиентах с тремя и более детьми недостаточно, что рассуждать об их кредитной истории с уверенностью.

<br/>

**По семейному положению**

In [58]:
create_pivot_table('family_status')

family_status,в разводе,вдовец / вдова,гражданский брак,женат / замужем,не женат / не замужем
0,1110,896,3775,11413,2536
1,85,63,388,931,274
2,7.11%,6.57%,9.32%,7.54%,9.75%


Не важно, одинок человек или не одинок и узаконены ли его отношения. Не столько семейное положение влияет на благонадёжность заёмщика, сколько его возраст.

<br/>

**По доходу**

In [59]:
create_pivot_table('income_id')

,very low,low,average,above average,high,very high,unknown
0,3280,3199,3100,3252,3141,3226,532
1,284,290,305,318,275,240,29
2,7.97%,8.31%,8.96%,8.91%,8.05%,6.92%,5.17%


Казалось, что клиенты с очень малым заработком будут очень часто иметь задолженности. Оказалось наоборот - у таких клиентов хорошие показатели. У клиентов с высоким уровнем дохода, ожидаемо, трудности с выплатой задолженности возникают реже. Наименее надёжны клиенты со средним заработком. Но явной взаимосвязи между доходами заёмщика и его надёжностью, судя по имеющимся данным, нет. Лучше ориентироваться на тип занятости, там закономерности налицо. 

<br/>

**По целям**

In [60]:
create_pivot_table('purpose_id')

purpose_id,bying a car,bying a place to live,bying a place to rent,education,home improvement,housebuilding,making a wedding
0,3905,5912,1813,3644,572,1735,2149
1,403,452,151,370,35,144,186
2,9.35%,7.10%,7.69%,9.22%,5.77%,7.66%,7.97%


Можно ли говорить, что существует взаимосвязь между целью, на которую взят кредит, и вероятностью его выплаты? Да, именно об этом говорят данные, хотя интерпретировать их можно по-разному. 

[К началу проекта](#target5)